### Imports

In [23]:
# Imports 
import numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn, os, math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


### Function for reading the csv files and dropping the unwanted columns

In [24]:
cols_to_drop = ['id', 'uri', 'duration_ms']

def read_csv_files(file_name, cols_to_drop=cols_to_drop): 
    df = pd.read_csv(file_name, index_col=0)
    df.drop(cols_to_drop, axis=1, inplace=True)
    
    labels = df['label']
    
    # This makes the categorical labels numbers. Print it to see... Better than labels.cat.codes
    y = labels.factorize()[0]
    
    # Add new axis to y in order to be able to transpose it when needed 
    # y = y[:, np.newaxis]
    return df, y

df, y = read_csv_files("data\csvs\dataframeV1.csv")

print(y)
df

[0 0 0 ... 3 3 3]


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,label
0,0.2750,0.1570,7,-18.752,1,0.0636,0.89000,0.842000,0.1860,0.3040,73.289,4,classic
1,0.2210,0.1260,0,-25.427,1,0.0447,0.98900,0.897000,0.1020,0.2160,133.630,4,classic
2,0.2890,0.0306,9,-30.790,0,0.0446,0.98700,0.911000,0.1020,0.1180,125.610,3,classic
3,0.0753,0.0700,2,-27.272,1,0.0440,0.91800,0.947000,0.1460,0.0625,79.801,4,classic
4,0.1300,0.1580,2,-16.132,1,0.0350,0.74800,0.924000,0.1000,0.0998,85.031,4,classic
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0.6510,0.7610,10,-7.801,1,0.2500,0.44600,0.000035,0.1110,0.8690,139.526,4,rap
46,0.8710,0.6390,7,-7.821,1,0.3490,0.12100,0.000000,0.1930,0.7640,141.060,4,rap
47,0.6170,0.4770,1,-9.889,1,0.3600,0.00422,0.000000,0.0830,0.4360,99.095,4,rap
48,0.8500,0.5640,1,-9.631,0,0.3830,0.23800,0.000000,0.1110,0.3480,139.920,4,rap


### Naive Bayes Classifier. 
Let's first get to know the calculations before the actual implementations.

Naive Bayes is a simple technique for constructing classifiers: models that assign class labels to problem instances, represented as vectors of feature values, where the class labels are drawn from some finite set. There is not a single algorithm for training such classifiers, but a family of algorithms based on a common principle: all naive Bayes classifiers assume that the value of a particular feature is independent of the value of any other feature, given the class variable. For example, a fruit may be considered to be an apple if it is red, round, and about 10 cm in diameter. A naive Bayes classifier considers each of these features to contribute independently to the probability that this fruit is an apple, regardless of any possible correlations between the color, roundness, and diameter features.

Abstractly, naive Bayes is a conditional probability model: given a problem instance to be classified, represented by a vector $ {\displaystyle \mathbf {x} =(x_{1},\ldots ,x_{n})}$ representing some n features (independent variables), it assigns to this instance probabilities
$$ {\LARGE{ \displaystyle p(C_{k}\mid x_{1},\ldots ,x_{n})\,}}$$
for each of K possible outcomes or classes ${\displaystyle C_{k}}.$


Using Bayes' theorem, the conditional probability can be decomposed as
$$ {\displaystyle p(C_{k}\mid \mathbf {x} )={\frac {p(C_{k})\ p(\mathbf {x} \mid C_{k})}{p(\mathbf {x} )}}\,} $$
In plain English, using Bayesian probability terminology, the above equation can be written as
$$ {\displaystyle {\text{posterior}}={\frac {{\text{prior}}\times {\text{likelihood}}}{\text{evidence}}}\,} $$

In practice, there is interest only in the numerator of that fraction, because the denominator does not depend on ${\displaystyle C}$ and the values of the features ${\displaystyle x_{i}}$ are given, so that the denominator is effectively constant. So the evidence,
$ {\displaystyle \sum _{k}p(C_{k})\ p(\mathbf {x} \mid C_{k})}$
is a scaling factor dependent only on ${\displaystyle x_{1},\ldots ,x_{n}}$, that is, a constant if the values of the feature variables are known. So we shall drop it since it does not influence the classification.

Again, in our case, since we have four classes each of which occuring with equal probability of $\frac{1}{4}$, we can see that the **prior**, $\mathbb{P}(\bold{C}_k)$, is effectively constant just like the evidence above. So, we shall drop it as well.


Now we have only the **likelihood**, $p(\mathbf {x} \mid C_{k})$ , to work with. The likelihood can be rewritten as follows, using the chain rule for repeated applications of the definition of conditional probability:

$$\displaystyle{p(x_1, x_2, \ldots, x_n \mid C_k) = p(x_{1}\mid x_{2},\ldots ,x_{n},C_{k})\ p(x_{2}\mid x_{3},\ldots ,x_{n},C_{k})\cdots p(x_{n-1}\mid x_{n},C_{k})\ p(x_{n}\mid C_{k})}$$

Now the **naive** conditional independence assumptions come into play: assume that all features in $\mathbf{x}$ are mutually independent, conditional on the category $\bold{C_{k}}$. Under this assumption,

$$\displaystyle {p(x_{i}\mid x_{i+1},\ldots ,x_{n},C_{k})=p(x_{i}\mid C_{k})\,}.$$

Thus, the joint model, after dropping the prior and the evidence, can be expressed as **proportional** to $\prod _{i=1}^{n}p(x_{i}\mid C_{k})$


Which Distribution should we use for the joint model **proportional** to $\prod _{i=1}^{n}p(x_{i}\mid C_{k})$? Even though there are too many distributions to use here, we opt for Gaussian Naive Bayes.

### Gaussian Naive Bayes.

When dealing with continuous data, a typical assumption is that the continuous values associated with each class are distributed according to a normal (or Gaussian) distribution. For example, suppose the training data contains a continuous attribute, $\bold{x}$. The data is first **segmented** by the class, and then the **mean** and **variance** of $\bold{x}$ is computed in each class. Let $\bold{\mu _{k}}$ be the mean of the values in $\bold{x}$ associated with class $\bold{C}_k $, and let $\bold{\sigma}_{k}^{2}$ be the unbiased sample variance of the values in $\bold{x}$ associated with class $\bold{C}_k $ (that is, the degree of freedom is 1 => n-1). Suppose one has collected some observation value v. Then, the probability density of v given a class $\bold{C}_k $, $\displaystyle{p(x=v\mid C_{k})}$, can be computed by plugging v into the equation for a normal distribution parameterized by $\displaystyle{\mu _{k}}$ and $\displaystyle{\sigma _{k}^{2}}$ as thus:


$$
\LARGE{ \mathbb{P}(\bold{X} = x | \mathbb{C_k}) = \frac{1}{\sqrt{2\pi\sigma_{k}^{2}}} \LARGE{e^{- \frac{(x-\mu_k)^2}{2\sigma_{k}^{2}}}} }

$$

In order to classify samples, one has to determine which posterior is greater: classic, jazz, metal or rap.

For example, for the classification as rap the posterior is given by

$$ posterior(rap) = \frac{\mathbb{P}(rap)\mathbb{P}(danceability|rap)...\mathbb{P}(tempo|rap)\mathbb{P}(timeSignature|rap)}{evidence} $$
where the

$$ \begin{align}
evidence &= \mathbb{P}(classic)\mathbb{P}(danceability|classic)...\mathbb{P}(timeSignature|classic) \\
        &+ \mathbb{P}(jazz)\mathbb{P}(danceability|jazz)...\mathbb{P}(timeSignature|jazz)\\
        &+ \mathbb{P}(metal)\mathbb{P}(danceability|metal)...\mathbb{P}(timeSignature|metal) \\
        &+ \mathbb{P}(rap)\mathbb{P}(danceability|rap)...\mathbb{P}(timeSignature|rap) 
\end{align}$$
and prior = $\frac{1}{4}$



But since we know that the evidence and the prior are effectively constants, we can drop them as already explained above. Thus:

- posterior(classic) = $\mathbb{P}(danceability|classic) \dots \mathbb{P}(tempo|classic)\mathbb{P}(timeSignature|classic)$
- posterior(jazz) = $\mathbb{P}(danceability|jazz)\dots \mathbb{P}(timeSignature|jazz)$
- posterior(metal) = $\mathbb{P}(danceability|metal)\dots \mathbb{P}(timeSignature|metal)$ 
- posterior(rap) = $\mathbb{P}(danceability|rap)\dots \mathbb{P}(tempo|rap)\mathbb{P}(timeSignature|rap)$

 We use the log-likelihood since the product of a large number of small probabilities can easily underflow the numerical precision of the computer. And this is resolved by computing instead the sum of the log probabilities as thus:

$$\begin{align}
\large{log\prod _{i=1}^{N}p(x_{i}\mid C_{k}) }
&=  \large{ \sum_{i=1}^N log \left( \mathbb{P}(\bold{X} = x_i | \mathbb{C_k} \right)} \\
&=  \large{ \sum_{i=1}^N log \left(\frac{1}{\sqrt{2\pi\sigma_{j,k}^{2}}}e^{- \large{ \frac{(x_i -\mu_{j,k})^2}{2\sigma_{j,k}^{2}}}} \right) }\\
&= \large{ \sum_{i=1}^N -\frac{1}{2}log \left(2\pi\sigma_{j,k}^2\right) - \frac{(x_i -\mu_{j,k})^2}{2\sigma_{j,k}^{2}} }
\end{align}$$

Where k is the class label and j is the index of the feature at column j

#### Getiing the MLE for the Mean and the Variance

```LOG-LIKELIHOOD```  =  $\bold{\ell}(\mu_{(j,k)}, \sigma_{(j,k)}^2) = -\frac{1}{2}\sum_{i=1}^N log(2\pi) -\sum_{i=1}^N log \left(\sigma_{j,k}\right) -\frac{1}{2} \sum_{i=1}^N \frac{(x_i -\mu_{j,k})^2}{\sigma_{j,k}^{2}}
$


```MEAN```:

$$ \begin{align}
\frac{\partial \ell}{\partial \mu_{j,k}} &= -\frac{1}{2} \sum_{i=1}^N \frac{2}{\sigma_{j,k}^2} (x_i -\mu_{j,k})(-1) = 0 \\
&\implies \sum_{i=1}^N \frac{x_i - \mu_{j,k}}{\sigma_{j,k}^2} = 0 \\
&\implies \frac{1}{\sigma_{j,k}^2} \sum_{i=1}^N x_i   -\frac{1}{\sigma_{j,k}^2} \sum_{i=1}^N \mu_{j,k} = 0 \\
&\implies \sum_{i=1}^N \mu_{j,k} = \sum_{i=1}^N x_i \\
&\implies \mu_{j,k} = \frac{1}{N}\sum_{i=1}^N x_i

\end{align}$$

```VARIANCE```:

$$\begin{align}
\frac{\partial \ell}{\partial \sigma_{j,k}^2} &= -\sum_{i=1}^N \frac{1}{\sigma_{j,k}} + \sum_{i=1}^N \frac{(x_i - \mu_{j,k})^2}{\sigma_{j,k}^3} = 0 \\
& \implies \frac{-N}{\sigma_{j,k}} + \frac{1}{\sigma_{j,k}^3} \sum_{i=1}^N (x_i - \mu_{j,k})^2 = 0 \\
& \implies \frac{1}{\sigma_{j,k}^3} \sum_{i=1}^N (x_i - \mu_{j,k})^2 = \frac{N}{\sigma_{j,k}} \\
& \implies \sigma_{j,k}^2 = \frac{1}{N} \sum_{i=1}^N (x_i - \mu_{j,k})^2
\end{align}$$


Therefore, the best estimate for the mean and variance parameters of a Gaussian are simply the empirical estimates of the mean and variance respectively. These means and variances are for each feature w.r.t each class. For example, the feature danceability w.r.t rap has a mean and variance different than danceability w.r.t classic. So for each feature x, it has a mean and variance for each class c.Therefore, the means and variances must be calculated for each feature (in our case, a whopping 12 features for 4 classes). We did this in a simple one-pass by first filtering out the appropriate instances (w.r.t to the class in question) under each feature, then take the mean and variance as can be seen in the implementation below.


### Implementation of the fit and predict methods

In [25]:

class GaussNaiveBayes(): 
    def __init__(self):
        self.mean_classic = None
        self.mean_jazz = None
        self.mean_rap = None
        self.mean_metal = None
        
        self.variance_classic = None
        self.variance_jazz = None
        self.variance_rap = None
        self.variance_metal = None
        
    ##############################################################################à
    def fit(self, X):
        # Notice that these are dictionaries, rather than lists
        self.mean_classic   = self.__preprocess(X, "classic").mean(numeric_only=True)  #mean_classic["danceability"]
        self.mean_jazz      = self.__preprocess(X, "jazz").mean(numeric_only=True)
        self.mean_metal     = self.__preprocess(X, "metal").mean(numeric_only=True)
        self.mean_rap       = self.__preprocess(X, "rap").mean(numeric_only=True)
        
        #degree of freedom = 1 => (n-1)
        self.variance_classic   = self.__preprocess(X, "classic").var(numeric_only=True, ddof=1)  
        self.variance_jazz      = self.__preprocess(X, "jazz").var(numeric_only=True, ddof=1)
        self.variance_metal     = self.__preprocess(X, "metal").var(numeric_only=True, ddof=1)
        self.variance_rap       = self.__preprocess(X, "rap").var(numeric_only=True, ddof=1)
        
    def __preprocess(self, X, labl):
        #index of all rows having label labl
        t = np.where(X.label == labl) 
        
        #values of those indices
        vals = X.iloc[t]     
        
        #drop the label because...          
        return vals.drop( ["label"], axis =1)   
    
    ######################################################################################à
    def predict(self, X_test):  #here, X_test without labels
        predictions = []
        if len(X_test) > 1:
            for row in X_test.values:
                pred = self.__prob_of_feature_given_label(row)
                m = np.argmax([pred["classic"], pred["jazz"], pred["metal"], pred["rap"]])
                predictions.append(m)
                
        #for testing single row
        elif len(X_test) == 1:
            pred = self.__prob_of_feature_given_label(X_test)
            m = np.argmax([pred["classic"], pred["jazz"], pred["metal"], pred["rap"]])
            predictions.append(m)
        
        return np.array(predictions)
        
    def __prob_of_feature_given_label(self, row):
        probs = {"classic": 0, "jazz": 0, "metal": 0, "rap": 0}
        log = math.log #in base e
        for i, val in enumerate(row):
            probs["classic"] += log( self.__gauss_func(val, self.mean_classic.iloc[i], self.variance_classic.iloc[i]))
            probs["jazz"]   +=  log( self.__gauss_func(val, self.mean_jazz.iloc[i], self.variance_jazz.iloc[i]) )
            probs["metal"]  +=  log( self.__gauss_func(val, self.mean_metal.iloc[i], self.variance_metal.iloc[i])) 
            probs["rap"]    +=  log( self.__gauss_func(val, self.mean_rap.iloc[i], self.variance_rap.iloc[i]) )
            #print(probs)
        return probs
        
    def __gauss_func(self, val, mu, sigma):
        v = (val-mu)**2
        s =  2*sigma**2
        euler = math.exp(-v/s)                        # e^(-(x-mean)^2 / 2sigma^2)
        scale = 1/math.sqrt(2*math.pi*sigma**2)       #1/sqrt(2pi*sigma^2)
        
        # return a very miniscule amount much closer to 0 than return 0 itself. This is useful for the log
        val = 1.21784378e-315 if (scale*euler)==0.0 else (scale*euler)
        #print(val)
        return val
        
        
        

### Train and Prediction

In [26]:
# Note that I didn't drop the label for the df
X_train, X_test, y_train, y_test = train_test_split(df, y, stratify=df["label"], shuffle=True, random_state=42)

# remove the label for X_test, not X_train
X_test = X_test.drop(["label"], axis=1)

GNB = GaussNaiveBayes()
GNB.fit(X_train)

y_pred = GNB.predict(X_test)
print(accuracy_score(y_pred, y_test))

for i in list( zip(y_test, y_pred)):
    print(i, end=' ')

0.8236994219653179
(1, 1) (2, 2) (2, 3) (0, 0) (1, 1) (2, 2) (1, 1) (3, 3) (3, 3) (2, 2) (2, 2) (3, 3) (3, 3) (1, 3) (2, 2) (2, 2) (1, 1) (1, 1) (2, 2) (0, 0) (3, 1) (3, 3) (3, 3) (1, 3) (1, 1) (0, 1) (2, 2) (3, 3) (2, 2) (3, 3) (1, 1) (3, 3) (0, 0) (3, 3) (2, 2) (3, 3) (2, 2) (3, 3) (0, 0) (3, 3) (3, 3) (2, 2) (3, 3) (2, 2) (1, 0) (2, 2) (1, 0) (1, 3) (3, 3) (2, 2) (3, 3) (2, 2) (2, 2) (0, 0) (0, 1) (2, 2) (2, 2) (1, 1) (2, 2) (3, 3) (1, 0) (2, 2) (3, 3) (0, 0) (3, 1) (1, 3) (2, 2) (1, 1) (2, 2) (1, 0) (0, 0) (0, 0) (3, 3) (1, 1) (3, 3) (3, 3) (1, 0) (2, 2) (1, 3) (0, 1) (0, 0) (0, 0) (2, 2) (1, 1) (1, 0) (1, 1) (3, 3) (1, 1) (0, 0) (1, 1) (0, 1) (1, 3) (0, 0) (0, 0) (3, 3) (3, 3) (2, 2) (2, 2) (3, 3) (0, 0) (2, 2) (1, 1) (1, 0) (2, 1) (0, 0) (2, 3) (0, 0) (1, 1) (2, 2) (3, 3) (1, 1) (2, 2) (3, 3) (3, 3) (3, 3) (1, 3) (1, 1) (3, 3) (3, 3) (2, 2) (1, 1) (2, 2) (0, 0) (3, 2) (2, 2) (2, 2) (2, 2) (0, 0) (1, 0) (1, 1) (1, 1) (1, 0) (0, 0) (2, 2) (1, 1) (2, 2) (0, 0) (0, 0) (0, 0) (2, 2) (